[ratsgo's blog](https://ratsgo.github.io/from%20frequency%20to%20semantics/2017/06/01/LDA/)  
[EnGear](http://www.engear.net/wp/topic-modeling-gensimpython/)  
[컴언-Ngram](http://blog.ilkyu.kr/entry/%EC%96%B8%EC%96%B4-%EB%AA%A8%EB%8D%B8%EB%A7%81-ngram)  
[LOVIT x DATA SCIENCE-lemmatization](https://lovit.github.io/nlp/2018/06/07/lemmatizer/)  
하기 내용은 상기 링크를 정리한것.

# Topic Modeling(Latent dirichlet Allocation)

In [1]:
import nltk; nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Andrew\AppData\L
[nltk_data]     ocal\conda\conda\envs\text3.5\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
!python -m spacy download en

C:\ProgramData\Anaconda3\python.exe: No module named spacy


In [4]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

In [9]:
# gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim # don't skip this
import matplotlib.pyplot as plt

%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level = logging.ERROR)

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [13]:
# 불용단어 로딩
# NLTK stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [15]:
df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
print(df.target_names.unique())
df.tail()

['rec.autos' 'comp.sys.mac.hardware' 'rec.motorcycles' 'misc.forsale'
 'comp.os.ms-windows.misc' 'alt.atheism' 'comp.graphics'
 'rec.sport.baseball' 'rec.sport.hockey' 'sci.electronics' 'sci.space'
 'talk.politics.misc' 'sci.med' 'talk.politics.mideast'
 'soc.religion.christian' 'comp.windows.x' 'comp.sys.ibm.pc.hardware'
 'talk.politics.guns' 'talk.religion.misc' 'sci.crypt']


,content,target,target_names
9995,From: sanjay@kin.lap.upenn.edu (Sanjay Sinha)\...,8,rec.motorcycles
9996,Organization: Central Michigan University\nFro...,7,rec.autos
9997,From: cr292@cleveland.Freenet.Edu (Jim Schenk)...,10,rec.sport.hockey
9998,From: wbdst+@pitt.edu (William B Dwinnell)\nSu...,1,comp.graphics
9999,From: ehung@ampex.com (Eric Hung)\nSubject: Re...,3,comp.sys.ibm.pc.hardware


In [20]:
# Convert to list
data = df.content.values.tolist()
# type(data)
# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters(공백제거)
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['From: (wheres my thing) Subject: WHAT car is this!? Nntp-Posting-Host: '
 'rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: '
 '15 I was wondering if anyone out there could enlighten me on this car I saw '
 'the other day. It was a 2-door sports car, looked to be from the late 60s/ '
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition, the front bumper was separate from the rest of the body. This is '
 'all I know. If anyone can tellme a model name, engine specs, years of '
 'production, where this car is made, history, or whatever info you have on '
 'this funky looking car, please e-mail. Thanks, - IL ---- brought to you by '
 'your neighborhood Lerxst ---- ']


## 단어 토큰화(형태소 분할)

In [21]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True)) # deacc = True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'edu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']]


## Bigram과 Trigram모델 만들기

In [27]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

C:\Users\Andrew\AppData\Local\conda\conda\envs\text3.5\lib\site-packages\gensim\models\phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [28]:
bigram['from']

C:\Users\Andrew\AppData\Local\conda\conda\envs\text3.5\lib\site-packages\gensim\models\phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['f', 'r', 'o', 'm']

In [29]:
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [32]:
# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp_posting_host', 'rac_wam_umd_edu', 'organization', 'university', 'of', 'maryland_college_park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front_bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']


## 불용단어 제거

In [33]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

In [34]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

In [35]:
def make_trigram(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

### 단어 원형 복원(lemmatization)
> studies = studi + es  
studying = study + ing

In [36]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """
    https://spacy.io/api/annotation
    """
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

함수 실행 파트

In [39]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do  lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [40]:
print(data_lemmatized[:1])

[['where', 's', 'thing', 'car', 'nntp_post', 'host', 'rac_wam', 'umd', 'organization', 'university', 'maryland_college', 'park', 'line', 'wonder', 'anyone', 'could', 'enlighten', 'car', 'see', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'bricklin', 'door', 'really', 'small', 'addition', 'front_bumper', 'separate', 'rest', 'body', 'know', 'anyone', 'tellme', 'model', 'name', 'engine', 'spec', 'year', 'production', 'car', 'make', 'history', 'whatev', 'info', 'funky', 'look', 'car', 'mail', 'thank', 'bring', 'neighborhood', 'lerxst']]


## 토픽 모델에 필요한 사전과 코퍼스 생성하기

LDA의 주요 입력값 코드를 생성할 것인데 그 두 가지 값은 사전(id2word)과 copus이다.

In [48]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

In [52]:
texts[0]

['where',
 's',
 'thing',
 'car',
 'nntp_post',
 'host',
 'rac_wam',
 'umd',
 'organization',
 'university',
 'maryland_college',
 'park',
 'line',
 'wonder',
 'anyone',
 'could',
 'enlighten',
 'car',
 'see',
 'day',
 'door',
 'sport',
 'car',
 'look',
 'late',
 'early',
 'call',
 'bricklin',
 'door',
 'really',
 'small',
 'addition',
 'front_bumper',
 'separate',
 'rest',
 'body',
 'know',
 'anyone',
 'tellme',
 'model',
 'name',
 'engine',
 'spec',
 'year',
 'production',
 'car',
 'make',
 'history',
 'whatev',
 'info',
 'funky',
 'look',
 'car',
 'mail',
 'thank',
 'bring',
 'neighborhood',
 'lerxst']

In [50]:
id2word.doc2bow(texts[0])

[(0, 1),
 (1, 2),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 5),
 (7, 1),
 (8, 1),
 (9, 2),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 1),
 (14, 1),
 (15, 1),
 (16, 1),
 (17, 1),
 (18, 1),
 (19, 1),
 (20, 1),
 (21, 1),
 (22, 2),
 (23, 1),
 (24, 1),
 (25, 1),
 (26, 1),
 (27, 1),
 (28, 1),
 (29, 1),
 (30, 1),
 (31, 1),
 (32, 1),
 (33, 1),
 (34, 1),
 (35, 1),
 (36, 1),
 (37, 1),
 (38, 1),
 (39, 1),
 (40, 1),
 (41, 1),
 (42, 1),
 (43, 1),
 (44, 1),
 (45, 1),
 (46, 1),
 (47, 1),
 (48, 1),
 (49, 1),
 (50, 1)]

In [55]:
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [56]:
# view
print(corpus[:1])

[[(0, 1), (1, 2), (2, 1), (3, 1), (4, 1), (5, 1), (6, 5), (7, 1), (8, 1), (9, 2), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 2), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1)]]


Gensim은 문서의 각 단어들마다 유니크한 ID를 생성한다.  
위의 튜플은 (word_id, word_frequency)이 매핑되어 있다.  
이것이 LDA모델의 입력으로 사용된다.  
아래는 아이디에 따른 단어를 확인해볼 수 있다.

In [57]:
id2word[0]

'addition'

또는 사람이 이해 가능한 형태의 코퍼스를 생성할 수 있다.   
(단어, 빈도 수)

In [60]:
# Human readable format of corpus (term-frequency)

[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('addition', 1),
  ('anyone', 2),
  ('body', 1),
  ('bricklin', 1),
  ('bring', 1),
  ('call', 1),
  ('car', 5),
  ('could', 1),
  ('day', 1),
  ('door', 2),
  ('early', 1),
  ('engine', 1),
  ('enlighten', 1),
  ('front_bumper', 1),
  ('funky', 1),
  ('history', 1),
  ('host', 1),
  ('info', 1),
  ('know', 1),
  ('late', 1),
  ('lerxst', 1),
  ('line', 1),
  ('look', 2),
  ('mail', 1),
  ('make', 1),
  ('maryland_college', 1),
  ('model', 1),
  ('name', 1),
  ('neighborhood', 1),
  ('nntp_post', 1),
  ('organization', 1),
  ('park', 1),
  ('production', 1),
  ('rac_wam', 1),
  ('really', 1),
  ('rest', 1),
  ('s', 1),
  ('see', 1),
  ('separate', 1),
  ('small', 1),
  ('spec', 1),
  ('sport', 1),
  ('tellme', 1),
  ('thank', 1),
  ('thing', 1),
  ('umd', 1),
  ('university', 1),
  ('whatev', 1),
  ('where', 1),
  ('wonder', 1),
  ('year', 1)]]

## 토픽 모델 생성

LDA모델 훈련을 위해서는 Copus, 사전, 토픽 개수를 제공해야한다.  
그 외에도, 알파와 에타는 토픽의 희소성에 영향을 주는 하이퍼파라미터 이다.  
Gensim docs에 따르면 기본값은 모두 '1.0/num_topics'을 기본값으로 사용한다.  <br>
<br>
chunksize는 각 훈련 chunk에서 사용할 문서의 수이다.   
update_every는 모델 매개변수를 업데이트 해야하는 빈도를 결정하고,  
passes는 총 훈련 과정 수를 결정한다.  

In [64]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word, 
                                            num_topics = 20, 
                                            random_state = 100, 
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto',
                                            per_word_topics=True)

## LDA 모델의 토픽 보기
위의 LDA모델은 각 토픽이 키워드의 조합이고,  
각 키워드가 토픽에 일정한 가중치를 부여하는 20개의 주제로 구성된다.  
`lda_model.print_topics()`를 사용하여 각 토픽의 키워드와 각 키워드의 중요도(가중치)를 볼 수 있다.

In [65]:
# Print the keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.034*"_" + 0.029*"blue" + 0.021*"tank" + 0.014*"cubs_suck" + 0.012*"eg" + '
  '0.012*"gas" + 0.011*"hi" + 0.007*"henry_spencer" + 0.007*"cigarette" + '
  '0.006*"xlib"'),
 (1,
  '0.020*"value" + 0.020*"bus" + 0.019*"specifically" + 0.015*"function" + '
  '0.015*"associate" + 0.013*"motorcycle" + 0.011*"properly" + 0.010*"code" + '
  '0.008*"confuse" + 0.008*"error"'),
 (2,
  '0.023*"window" + 0.020*"card" + 0.017*"file" + 0.014*"drive" + 0.013*"use" '
  '+ 0.013*"system" + 0.011*"problem" + 0.010*"run" + 0.009*"color" + '
  '0.009*"do"'),
 (3,
  '0.035*"game" + 0.033*"team" + 0.019*"player" + 0.017*"play" + 0.017*"win" + '
  '0.016*"hockey" + 0.013*"season" + 0.011*"contact" + 0.011*"year" + '
  '0.010*"nhl"'),
 (4,
  '0.026*"wire" + 0.017*"circuit" + 0.015*"faq" + 0.015*"connect" + '
  '0.013*"wiring" + 0.013*"voice" + 0.013*"cover" + 0.011*"outlet" + '
  '0.011*"neutral" + 0.010*"conference"'),
 (5,
  '0.030*"government" + 0.024*"gun" + 0.022*"law" + 0.017*"state" + '
  '0.0

## 해석 방법  
```
[(0,
  '0.034*"_" + 0.029*"blue" + 0.021*"tank" + 0.014*"cubs_suck" + 0.012*"eg" + '
  '0.012*"gas" + 0.011*"hi" + 0.007*"henry_spencer" + 0.007*"cigarette" + '
  '0.006*"xlib"'),
```
토픽0 는 위와 같이 표현된다.  
이 토픽에 대해 기여하고 있는 상위 10개 키워드는 'blue', 'tank', 'cubs_suck'등 이며,  
토픽 0에 대한 'blue'의 가중치는 0.029이다.  
가중치는 키워드가 해당 토픽에 얼마나 중요하는지를 나타낸다.  
다시말하자면 토픽 0는 blue나 tank일 수 있다.  

## 모델 난이도와 일관성 점수 계산

In [68]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus)) # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts = data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.754099497677107

Coherence Score:  0.515758393754623


## 토픽-키워드 시각화 

In [69]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

C:\Users\Andrew\AppData\Local\conda\conda\envs\text3.5\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
7      16.239773        1       1  0.336833 -0.001198
13     15.221648        1       2  0.291034 -0.003244
18     14.588356        1       3  0.328827 -0.128122
2       6.161590        1       4  0.091782  0.226447
15      5.973034        1       5  0.059636  0.222487
10      4.344124        1       6 -0.085009 -0.113136
17      4.177513        1       7  0.060714 -0.183179
12      3.604345        1       8 -0.021421 -0.036566
5       3.365388        1       9 -0.022882 -0.021291
16      3.271580        1      10 -0.029247  0.219681
6       3.222101        1      11 -0.063057 -0.016647
9       2.968343        1      12 -0.062890  0.139384
14      2.739964        1      13 -0.088897 -0.062678
3       2.731095        1      14 -0.047749 -0.117440
8       2.291429        1      15 -0.111444  0.003718
19      2.127126        1      16 -0.113703 -0.023427
11      2.077360        1      17 -0.092439 -0.110459
4       1.694270        1      18 -0.134219  0.001629
0       1.679200        1      19 -0.164229 -0.027983
1       1.521758        1      20 -0.131640  0.032026, topic_info=      Category          Freq          Term         Total  loglift  logprob
term                                                                      
58974  Default  57084.000000            ax  57084.000000  30.0000  30.0000
50035  Default  12491.000000          line  12491.000000  29.0000  29.0000
3119   Default  16290.000000           not  16290.000000  28.0000  28.0000
5951   Default  11142.000000  organization  11142.000000  27.0000  27.0000
62146  Default   3844.000000           max   3844.000000  26.0000  26.0000
30601  Default  10452.000000            do  10452.000000  25.0000  25.0000
23211  Default   9655.000000         write   9655.000000  24.0000  24.0000
5339   Default   7586.000000       article   7586.000000  23.0000  23.0000
37371  Default   8046.000000            be   8046.000000  22.0000  22.0000
65832  Default  11782.000000         would  11782.000000  21.0000  21.0000
39150  Default   6905.000000           say   6905.000000  20.0000  20.0000
63353  Default   5803.000000            go   5803.000000  19.0000  19.0000
40779  Default   4407.000000     nntp_post   4407.000000  18.0000  18.0000
36239  Default   4083.000000          host   4083.000000  17.0000  17.0000
38228  Default   6852.000000           get   6852.000000  16.0000  16.0000
17778  Default   5027.000000    university   5027.000000  15.0000  15.0000
32058  Default   1863.000000         space   1863.000000  14.0000  14.0000
48102  Default   2294.000000        window   2294.000000  13.0000  13.0000
33661  Default   3735.000000             s   3735.000000  12.0000  12.0000
60576  Default   7291.000000          know   7291.000000  11.0000  11.0000
52514  Default   3461.000000          year   3461.000000  10.0000  10.0000
40213  Default   4598.000000           use   4598.000000   9.0000   9.0000
36322  Default   1540.000000          game   1540.000000   8.0000   8.0000
70022  Default   1948.000000          mail   1948.000000   7.0000   7.0000
47172  Default   3005.000000         right   3005.000000   6.0000   6.0000
20102  Default   1685.000000     christian   1685.000000   5.0000   5.0000
65033  Default   1746.000000    government   1746.000000   4.0000   4.0000
52133  Default   1464.000000          team   1464.000000   3.0000   3.0000
21941  Default   3380.000000        system   3380.000000   2.0000   2.0000
38200  Default   2012.000000          card   2012.000000   1.0000   1.0000
...        ...           ...           ...           ...      ...      ...
28185  Topic20    119.369888          guru    120.321815   4.1774  -5.3259
40363  Topic20    117.622414  countersteer    118.574341   4.1772  -5.3407
71609  Topic20    113.025116       bradley    113.977043   4.1769  -5.3806
14846  Topic20    112.215775          lane    113.167702   4.1769  -5

원이 클수록 그 토픽이 더 일반적이다.  
즉, 특성화 하지 않고 여러 단어가 등장한다는 의미이다.  
좋은 토픽모델은 하나의 사분면에 모두 군집화 되지 않고 차트 전체적으로 흩어진 상태가 가장 좋다.